In [136]:
import pandas as pd
import numpy as np

#names=['Datum(UTC)','Nicht Erneuerbar (MW)', 'Erneuerbar (MW)', 'DE-LU Day Ahead EUR/MWh']
Wetter_Eki=pd.read_csv("Wetter/2024_Stunde_117_Ehrenkirchen.csv",sep=';',decimal=",",header=0,na_values='-').fillna(0)

In [137]:
#Stromfluss['Datum']=pd.to_datetime(Stromfluss['Datum von'],format='%d.%m.%Y %H:%M')
#Stromfluss['Datum']=Stromfluss['Datum'].dt.tz_localize('Europe/Berlin',ambiguous='infer')
#Stromfluss['Date']=Stromfluss['Datum'].dt.tz_convert('UTC')
Wetter_Eki['Datum']=pd.to_datetime(Wetter_Eki['Tag']+' '+Wetter_Eki['Stunde']+' +0100',format='%d.%m.%Y %H:%M %z',utc=True)
#Wetter_Eki['Datum']=Wetter_Eki['Datum'].dt.tz_localize('Europe/Berlin',ambiguous='infer')
Wetter_Eki=Wetter_Eki.set_index('Datum')

In [138]:
DWD_Daten = pd.read_csv("stundenwerte_FF_02115_19590101_20231231_hist/produkt_ff_stunde_20230727_20250126_02115.txt",sep=';',na_values='-999').fillna(0)
DWD_Daten['Datum']=pd.to_datetime(DWD_Daten['MESS_DATUM'],format='%Y%m%d%H',utc=True)
DWD_Daten=DWD_Daten.set_index('Datum')

In [139]:
names=['Datum(UTC)','Nicht Erneuerbar (MW)', 'Erneuerbar (MW)', 'DE-LU Day Ahead EUR/MWh']
DayAheadPrices=pd.read_csv("Daten entsoe/energy-charts_Stromproduktion_und_Börsenstrompreise_in_Deutschland_2024.csv",skiprows=2,names=names)
DayAheadPrices['Datum']=pd.to_datetime(DayAheadPrices['Datum(UTC)'])
DayAheadPrices=DayAheadPrices.set_index('Datum')
preis_menge=DayAheadPrices
preis_menge[['AVG_TA200','AVG_WV200','MAX_WV200max','SUM_GS200']]=Wetter_Eki[['AVG_TA200','AVG_WV200','MAX_WV200max','SUM_GS200']]
preis_menge['Wind Helgoland']=DWD_Daten['   F']

In [140]:
preis_menge['AVG_WV200'].mean(),preis_menge['MAX_WV200max'].mean(),preis_menge['Wind Helgoland'].mean()

(1.5477345173041894, 3.3853483606557377, 6.552277409018083)

In [141]:
Windkraft_je_ms=pd.read_csv('Windanlagen_kw_pro_ms.csv',sep=';')
Windkraft_je_ms['max W/m²']=Windkraft_je_ms.apply(lambda row: pow(min(row['m/s'],15),3)*0.5*1.2, axis= 1)
Windkraft_je_ms['m/s cp']=Windkraft_je_ms['m/s']
Windkraft_je_ms.set_index('m/s',drop=True,inplace=True)
Windkraft_je_ms['rel_V172-7.2']=Windkraft_je_ms['V172-7.2']/Windkraft_je_ms['V172-7.2'].max()
Windkraft_je_ms.head()


,V162-5.6,V162-6.2,V172-7.2,V172-7.2SO,EK100,1998_600kw,E-126,Kitepower_Hawk,Kitepower_Falcon,SkySails_PN-14,max W/m²,m/s cp,rel_V172-7.2
m/s,,,,,,,,,,,,,
0.0,0,0,0,0,0.0,0,0,-7.0,-4.0,0.0,0.000,0.0,0.0
0.5,0,0,0,0,0.0,0,0,-7.0,-4.0,0.0,0.075,0.5,0.0
1.0,0,0,0,0,0.0,0,0,-6.5,-3.0,0.0,0.600,1.0,0.0
1.5,0,0,0,0,0.0,0,0,-6.0,-2.0,0.0,2.025,1.5,0.0
2.0,0,0,0,0,0.0,0,0,-5.5,-0.5,0.0,4.800,2.0,0.0


In [161]:
faktor=5.7
# Basis Eki
basis=preis_menge['MAX_WV200max']
# Basis Helgoland
#basis=preis_menge['Wind Helgoland']
#faktor=11.6
fak='{}'.format(faktor)
fak_halbe='{}'.format(faktor/2)
max_wind=500
preis_menge['Wind '+fak+' m/s']=round(faktor*basis/basis.mean()*2)/2
preis_menge['Wind '+fak+' m/s']=preis_menge.apply(lambda row: min(row['Wind '+fak+' m/s'],max_wind),axis= 1)

preis_menge['max W/m²']=preis_menge.apply(lambda row: pow(min(row['Wind '+fak+' m/s'],15),3)*0.5*1.2, axis= 1)
preis_menge['pow H'+fak+' V172']=preis_menge['Wind '+fak+' m/s'].map(Windkraft_je_ms['rel_V172-7.2'])
preis_menge['Wert H'+fak+' V172']=preis_menge['pow H'+fak+' V172']*preis_menge['DE-LU Day Ahead EUR/MWh']/1000
preis_menge['z Anzahl']=1

In [162]:
pm_pv=preis_menge['SUM_GS200']
pm_preis=preis_menge['DE-LU Day Ahead EUR/MWh']
pm_wind=preis_menge['Wind '+fak+' m/s']

preis_menge['kat']=(np.where(pm_preis<0,'0: <0 ct/kWh',np.where(pm_pv>300,'1: PV > 300 W/m2',np.where(pm_preis<35,'2: <3,5 ct/kWh',np.where(pm_wind<faktor/2,'3: Wind < '+fak_halbe,np.where(pm_wind<faktor,'4: Wind < '+fak,'9: Rest'))))))

In [163]:
preis_menge['Wind '+fak+' m/s'].mean(),preis_menge['max W/m²'].mean(),preis_menge['pow H'+fak+' V172'].sum(),preis_menge['Wert H'+fak+' V172'].sum()

(5.677823315118397, 252.2981045081967, 2103.489722222222, 132.64719511805555)

In [164]:
preis_menge['Monat']=preis_menge.index.month
preis_menge['Jahr']=preis_menge.index.year
Strom=pd.pivot_table(preis_menge,index='Monat', columns=['kat'],values=['Wert H'+fak+' V172','pow H'+fak+' V172','z Anzahl'], aggfunc='sum',margins=True,margins_name='2024')

In [165]:
Strom

Wert H5.7 V172                                                 \
kat     0: <0 ct/kWh 1: PV > 300 W/m2 2: <3,5 ct/kWh 3: Wind < 2.85   
Monat                                                                 
1          -0.016300         0.203871       0.319252            0.0   
2          -0.000280         1.128826       0.505976            0.0   
3          -0.014436         1.769830       0.336139            0.0   
4          -0.598645         2.643492       0.275002            0.0   
5          -0.229443         1.979697       0.236623            0.0   
6          -0.356173         3.065271       0.052289            0.0   
7          -0.200685         1.017329       0.062966            0.0   
8          -0.105274         1.049623       0.021504            0.0   
9          -0.018706         0.816191       0.233363            0.0   
10         -0.005689         0.313994       0.166366            0.0   
11         -0.001415         0.393710       0.159304            0.0   
12         -0.004847         0.128324       0.846565            0.0   
2024       -1.551892        14.510157       3.215348            0.0   

                                            pow H5.7 V172                   \
kat   4: Wind < 5.7     9: Rest        2024  0: <0 ct/kWh 1: PV > 300 W/m2   
Monat                                                                        
1          1.038125   17.427243   18.972191     15.615972         3.269167   
2          0.698445   16.402228   18.735194      4.000000        26.991806   
3          0.982607    9.973575   13.047715      3.983333        37.276389   
4          1.133174   11.453310   14.906334     28.628472        53.761944   
5          1.070332    6.979724   10.036933     21.028194        30.973750   
6          1.435002    2.768753    6.965143     18.617778        45.916806   
7          0.916749    1.458721    3.255081     15.517500        19.559722   
8          1.000879    1.332993    3.299724     11.675694        16.815972   
9          1.053080    5.351538    7.435465     11.407361        17.924861   
10         1.249957    3.151161    4.875789      7.542361         4.867639   
11         1.346830   14.212962   16.111391      1.568611         4.038056   
12         1.329706   12.706487   15.006235      7.988472         1.181389   
2024      13.254887  103.218695  132.647195    147.573750       262.577500   

                      ...                                          \
kat   2: <3,5 ct/kWh  ... 4: Wind < 5.7      9: Rest         2024   
Monat                 ...                                           
1          30.763056  ...     12.010139   232.509861   294.168194   
2          38.881944  ...     10.222917   250.802222   330.898889   
3          26.526111  ...     12.994861   143.059722   223.840417   
4          32.135139  ...     13.495556   149.649167   277.670278   
5          15.007778  ...     12.456806    81.950556   161.417083   
6           8.891806  ...     12.244028    30.484306   116.154722   
7           6.543611  ...     10.004444    15.949722    67.575000   
8           1.772083  ...      9.239861    13.042639    52.546250   
9          17.322361  ...     10.944306    66.775972   124.374861   
10          9.068889  ...     12.910556    38.615417    73.004861   
11          9.854444  ...     10.030000   137.113472   162.604583   
12         57.492083  ...     10.045278   142.527361   219.234583   
2024      254.259306  ...    136.598750  1302.480417  2103.489722   

          z Anzahl                                                 \
kat   0: <0 ct/kWh 1: PV > 300 W/m2 2: <3,5 ct/kWh 3: Wind < 2.85   
Monat                                                               
1             16.0             19.0           32.0          108.0   
2              4.0             52.0           47.0           42.0   
3             12.0            100.0           51.0          120.0   
4             50.0            115.0           67.0           31.0   
5             78.0            11

'5.7'

,STATIONS_ID,MESS_DATUM,QN_3,F,D,eor
0,2115,2023072700,10,6.4,260,eor
1,2115,2023072701,10,6.9,250,eor
2,2115,2023072702,10,6.7,240,eor
3,2115,2023072703,10,6.1,230,eor
4,2115,2023072704,10,5.5,230,eor


In [124]:
DWD_Daten['   F'].mean()

5.836610797868232

In [127]:
DWD_Daten

,STATIONS_ID,MESS_DATUM,QN_3,F,D,eor,Datum
0,2115,2023072700,10,6.4,260,eor,2023-07-27 00:00:00+00:00
1,2115,2023072701,10,6.9,250,eor,2023-07-27 01:00:00+00:00
2,2115,2023072702,10,6.7,240,eor,2023-07-27 02:00:00+00:00
3,2115,2023072703,10,6.1,230,eor,2023-07-27 03:00:00+00:00
4,2115,2023072704,10,5.5,230,eor,2023-07-27 04:00:00+00:00
...,...,...,...,...,...,...,...
12942,2115,2025012619,1,5.9,160,eor,2025-01-26 19:00:00+00:00
12943,2115,2025012620,1,7.1,160,eor,2025-01-26 20:00:00+00:00
12944,2115,2025012621,1,8.5,140,eor,2025-01-26 21:00:00+00:00
12945,2115,2025012622,1,9.9,130,eor,2025-01-26 22:00:00+00:00
